In [1]:
using GLPK
using JuMP

In [2]:
lines = readlines("data/VA01.dat")
k = 3 #instance 1
n = parse(Int,lines[1]) 
ci = [parse(Int,x) for x in split(lines[2])]
cp = fill(0, n, n)
for i in 1:(n-1)
    currentLine = [parse(Int,x) for x in split(lines[2+i])]
    for j in 1:length(currentLine)
        cp[i,i+j] = currentLine[j]
        cp[i+j,i] = currentLine[j]
    end
end
p = [parse(Int,x) for x in split(lines[n + 5])]
P = fill(0.8*sum(p)/k,k);

In [3]:
m = Model();

In [4]:
set_optimizer(m, GLPK.Optimizer);

In [5]:
@variable(m, v[1:k, 1:n], binary=true);

In [6]:
@variable(m, a[1:n, 1:n], binary=true);

In [22]:
@objective(m, Max, sum(sum(ci[j]*v[i,j] for j in 1:n) for i in 1:k) + sum(sum(sum(cp[o,j] * ((v[i,o]+v[i,j])/2) for o in 1:n) for j in 1:n) for i in 1:k))

1517 v[1,4] + 945 v[1,5] + 1322 v[1,6] + 1316 v[1,11] + 1670 v[1,12] + 1229 v[1,13] + 948 v[1,15] + 1758 v[1,18] + 1548 v[1,22] + 1289 v[1,25] + 986 v[1,34] + 1569 v[1,35] + 1207 v[1,40] + 1317 v[1,41] + 1346 v[1,42] + 1306 v[1,43] + 1352 v[1,57] + 1000 v[1,58] + 1471 v[1,59] + 1192 v[1,71] + 1293 v[1,73] + 1035 v[1,75] + 1620 v[1,80] + 1729 v[1,82] + 2009 v[1,91] + 1253 v[1,93] + 980 v[1,94] + 1716 v[1,100] + 1517 v[2,4] + 945 v[2,5] + 1322 v[2,6] + 1316 v[2,11] + 1670 v[2,12] + 1229 v[2,13] + 948 v[2,15] + 1758 v[2,18] + 1548 v[2,22] + 1289 v[2,25] + 986 v[2,34] + 1569 v[2,35] + 1207 v[2,40] + 1317 v[2,41] + 1346 v[2,42] + 1306 v[2,43] + 1352 v[2,57] + 1000 v[2,58] + 1471 v[2,59] + 1192 v[2,71] + 1293 v[2,73] + 1035 v[2,75] + 1620 v[2,80] + 1729 v[2,82] + 2009 v[2,91] + 1253 v[2,93] + 980 v[2,94] + 1716 v[2,100] + 1517 v[3,4] + 945 v[3,5] + 1322 v[3,6] + 1316 v[3,11] + 1670 v[3,12] + 1229 v[3,13] + 948 v[3,15] + 1758 v[3,18] + 1548 v[3,22] + 1289 v[3,25] + 986 v[3,34] + 1569 v[3,35] 

In [8]:
@constraint(m, [j in n], sum(v[i,j] for i in k) <= 1)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 100
And data, a 1-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 v[3,100] ≤ 1.0

In [9]:
@constraint(m, [i in k], sum(p[j] * v[i,j] for j in n) <= P[i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 3
And data, a 1-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 14 v[3,100] ≤ 688.5333333333333

In [23]:
optimize!(m)

In [24]:
println("Valor ótimo:  $(JuMP.objective_value(m))")

Valor ótimo:  389289.0


In [12]:
println(sum(value())

52.0
